# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [ ]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [ ]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(structure_length / offset_x) + 1
Ny = int(structure_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [13]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [ ]:
# mapdl.open_gui()

## Step 2 : Generate Geometry

In [5]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('SI')      # SI unit system

## Create geometry of honeycomb core
mapdl.block(0, structure_length, 0, structure_breadth, 0, node_length)
for i in range(0,Nx):                                                                                   ## column number
    for j in range(0,Ny):
        if j%2 == 0:                                                                                    ## even row number
            mapdl.rpr4(6, i*offset_x , j*offset_y, L, 90, node_length)
        else:                                                                                           ## odd row number
            mapdl.rpr4(6, i*offset_x + offset_x/2 , j*offset_y, L, 90, node_length)
mapdl.vsbv(1,'all')
honeycomb_core = mapdl.cm("CORE", "VOLU")

## Generate geometry of facesheets
front_facesheet = mapdl.block(0, structure_length, 0, structure_breadth, node_length, node_length + facesheet_thickness)  ## Top facesheet
rear_facesheet = mapdl.block(0, structure_length, 0, structure_breadth, -1*facesheet_thickness, 0)                        ## Bottom facesheet

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = node_length + facesheet_thickness + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# Generate Geometry of 0.44 Remington Magnum Bullet
mapdl.cone(0.0052, 0.005486, initial_distance , initial_distance + 0.00825)
mapdl.cone(0.00569, 0.005805, initial_distance + 0.00825 , initial_distance + 0.04089 )
mapdl.vadd(3,4)
bullet = mapdl.cm("BULLET","VOLU")

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties, mesh attributes and generate mesh

In [6]:
## Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

## Define mesh controls

mapdl.lesize("ALL", 0.001, layer1=1)
# mapdl.smrtsize(1)

# Mesh facesheets
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('2')
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('1')

# Mesh honeycomb core
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('CORE')

# Mesh bullet
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('BULLET')

## Plot the mesh
mapdl.eplot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 4 - apply loads and boundary conditions

In [7]:
### Make the lateral sides of the panel as fixed support i.e. set displacement of nodes to zero

# Select all nodes at the x = 0 end (plane) of the sandwich panel
mapdl.nsel("S", "LOC", "X", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the x = structure_length end of the sandwich panel
mapdl.nsel("S", "LOC", "X", structure_length)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = structure_breadth end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", structure_breadth)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")



SPECIFIED CONSTRAINT UX   FOR SELECTED NODES            1 TO      181204 BY           1
 REAL=  0.00000000       IMAG=  0.00000000    
 ADDITIONAL DOFS=  UY    UZ

In [8]:
mapdl.edpvel('VELO', 'BULLET', vz='-400')

MapdlRuntimeError: 

Error in instance GRPC_127.0.0.1:50052

*** ERROR ***                           CP =      24.453   TIME= 16:20:42
 The capability LsDyna was either not ordered with this Ansys            
 installation or the appropriate product was not selected for this       
 session.                                                                
  Contact your Ansys support person for more information.

In [11]:
# from ansys.mapdl.core import Report
# print(Report())

-------------------------------------------------------------------------------


PyMAPDL Software and Environment Report


Packages Requirements
*********************

Core packages
-------------
ansys.mapdl.core    : 0.64.0
numpy               : 1.24.3
appdirs             : 1.4.4
scipy               : 1.10.1
grpc                : Package not found
ansys.api.mapdl.v0  : Package not found
ansys.mapdl.reader  : 0.52.13
google.protobuf     : Package not found

Optional packages
-----------------
matplotlib          : 3.6.2
pyvista             : 0.37.0
pyiges              : 0.2.1
tqdm                : 4.65.0


Ansys Installation
******************
Version   Location
------------------
231       C:\Program Files\ANSYS Inc\v231


Ansys Environment Variables
***************************
ANSYS231_DIR                   C:\Program Files\ANSYS Inc\v231\ANSYS
ANSYSLIC_DIR                   C:\Program Files\ANSYS Inc\Shared Files\Licensing
AWP_LOCALE231                  en-us
AWP_ROOT231           

In [ ]:
# mapdl.ignore_errors = True

In [ ]:
# mapdl.run("/SOLU")
# mapdl.antype(4)  # transient analysis
# mapdl.trnopt("FULL")  # full transient analysis
# mapdl.run("/STATUS,SOLU")
# mapdl.solve()

In [14]:
# mapdl.exit()